# Data import

This is a short demonstration that shows you how you can import raw data into the Neo4j database without tools like jQAssistant (because you cannot use them in your programming language).

## Import dependencies (from jdeps)

Import dataset

In [1]:
import pandas as pd

deps = pd.read_csv("data/spring_petclinic_deps.txt", names=["raw"], sep="\n")
deps.head()

,raw
0,spring-petclinic-2.4.5.jar -> not found
1,spring-petclinic-2.4.5.jar -> /usr/lib/jvm/jav...
2,org.springframework.boot.loader.ClassPathIn...
3,org.springframework.boot.loader.ClassPathIn...
4,org.springframework.boot.loader.ClassPathIn...


Normalize data

In [2]:
# class entries begin with three whitespaces
deps = deps[deps['raw'].str.startswith("   ")]
# separates the source from the target
splitted = deps['raw'].str.split("->", n=1, expand=True)
# remove whitespaces from source
deps['from'] = splitted[0].str.strip()
# get the target and the artifact names
splitted_2 = splitted[1].str.split(" ", n=2)
deps['to'] = splitted_2.str[1]
deps['type'] = splitted_2.str[2].str.strip()
deps['name'] = deps['from'].str.split(".").str[-1]
deps.head()

,raw,from,to,type,name
2,org.springframework.boot.loader.ClassPathIn...,org.springframework.boot.loader.ClassPathIndex...,java.io.BufferedReader,,ClassPathIndexFile
3,org.springframework.boot.loader.ClassPathIn...,org.springframework.boot.loader.ClassPathIndex...,java.io.File,,ClassPathIndexFile
4,org.springframework.boot.loader.ClassPathIn...,org.springframework.boot.loader.ClassPathIndex...,java.io.FileInputStream,,ClassPathIndexFile
5,org.springframework.boot.loader.ClassPathIn...,org.springframework.boot.loader.ClassPathIndex...,java.io.IOException,,ClassPathIndexFile
6,org.springframework.boot.loader.ClassPathIn...,org.springframework.boot.loader.ClassPathIndex...,java.io.InputStream,,ClassPathIndexFile


Focus on core application

In [3]:
petclinic_deps = deps[
    (
        deps['from'].str.startswith("org.springframework.samples.petclinic") |
        deps['to'].str.startswith("org.springframework.samples.petclinic")
    )
    ].copy()
petclinic_deps.head()

,raw,from,to,type,name
867,org.springframework.samples.petclinic.PetCl...,org.springframework.samples.petclinic.PetClini...,java.lang.Class,,PetClinicApplication
868,org.springframework.samples.petclinic.PetCl...,org.springframework.samples.petclinic.PetClini...,java.lang.Object,,PetClinicApplication
869,org.springframework.samples.petclinic.PetCl...,org.springframework.samples.petclinic.PetClini...,java.lang.String,,PetClinicApplication
870,org.springframework.samples.petclinic.PetCl...,org.springframework.samples.petclinic.PetClini...,org.springframework.boot.SpringApplication,not found,PetClinicApplication
871,org.springframework.samples.petclinic.PetCl...,org.springframework.samples.petclinic.PetClini...,org.springframework.boot.autoconfigure.SpringB...,not found,PetClinicApplication


## Import coverage data (from JaCoCo)

Import dataset

In [4]:
coverage = pd.read_csv("data/spring_petclinic_production_coverage_data.csv")
coverage.head()

,PACKAGE,CLASS,LINE_MISSED,LINE_COVERED
0,org.springframework.samples.petclinic,PetclinicInitializer,0,24
1,org.springframework.samples.petclinic.model,NamedEntity,1,4
2,org.springframework.samples.petclinic.model,Specialty,0,1
3,org.springframework.samples.petclinic.model,PetType,0,1
4,org.springframework.samples.petclinic.model,Vets,4,0


Enrich data

In [5]:
coverage['lines'] = coverage.LINE_COVERED + coverage.LINE_MISSED
coverage['ratio'] = coverage.LINE_COVERED / coverage.lines
coverage.head()

,PACKAGE,CLASS,LINE_MISSED,LINE_COVERED,lines,ratio
0,org.springframework.samples.petclinic,PetclinicInitializer,0,24,24,1.0
1,org.springframework.samples.petclinic.model,NamedEntity,1,4,5,0.8
2,org.springframework.samples.petclinic.model,Specialty,0,1,1,1.0
3,org.springframework.samples.petclinic.model,PetType,0,1,1,1.0
4,org.springframework.samples.petclinic.model,Vets,4,0,4,0.0


Normalize data

In [6]:
coverage['fqn'] = coverage["PACKAGE"] + "." + coverage["CLASS"]
coverage.head()

,PACKAGE,CLASS,LINE_MISSED,LINE_COVERED,lines,ratio,fqn
0,org.springframework.samples.petclinic,PetclinicInitializer,0,24,24,1.0,org.springframework.samples.petclinic.Petclini...
1,org.springframework.samples.petclinic.model,NamedEntity,1,4,5,0.8,org.springframework.samples.petclinic.model.Na...
2,org.springframework.samples.petclinic.model,Specialty,0,1,1,1.0,org.springframework.samples.petclinic.model.Sp...
3,org.springframework.samples.petclinic.model,PetType,0,1,1,1.0,org.springframework.samples.petclinic.model.Pe...
4,org.springframework.samples.petclinic.model,Vets,4,0,4,0.0,org.springframework.samples.petclinic.model.Vets


## Import source code data

Import data

In [7]:
cloc = pd.read_csv("data/spring_petclinic_cloc.csv")[:-1].copy()
cloc.tail()

,language,filename,blank,comment,code,"github.com/AlDanial/cloc v 1.82 T=0.33 s (75.4 files/s, 4843.0 lines/s)"
20,Java,./org/springframework/samples/petclinic/vet/Sp...,5,20,9,NaN
21,Java,./org/springframework/samples/petclinic/visit/...,6,31,9,NaN
22,Java,./org/springframework/samples/petclinic/PetCli...,5,21,9,NaN
23,Java,./org/springframework/samples/petclinic/owner/...,4,18,8,NaN
24,Java,./org/springframework/samples/petclinic/model/...,1,18,1,NaN


Normalize data

In [8]:
cloc['fqn'] = cloc['filename'].str.replace("./", "", regex=False)\
                              .str.replace("/",".", regex=False)\
                              .str.replace(".java","", regex=False)
cloc.head()

,language,filename,blank,comment,code,"github.com/AlDanial/cloc v 1.82 T=0.33 s (75.4 files/s, 4843.0 lines/s)",fqn
0,Java,./org/springframework/samples/petclinic/owner/...,18,31,96,NaN,org.springframework.samples.petclinic.owner.Ow...
1,Java,./org/springframework/samples/petclinic/owner/...,23,33,94,NaN,org.springframework.samples.petclinic.owner.Owner
2,Java,./org/springframework/samples/petclinic/owner/...,16,20,77,NaN,org.springframework.samples.petclinic.owner.Pe...
3,Java,./org/springframework/samples/petclinic/owner/...,19,22,71,NaN,org.springframework.samples.petclinic.owner.Pet
4,Java,./org/springframework/samples/petclinic/owner/...,12,31,49,NaN,org.springframework.samples.petclinic.owner.Vi...


In [9]:
loc = cloc[['fqn', 'code', 'comment', 'blank']].dropna().copy()
loc.head()

,fqn,code,comment,blank
0,org.springframework.samples.petclinic.owner.Ow...,96,31,18
1,org.springframework.samples.petclinic.owner.Owner,94,33,23
2,org.springframework.samples.petclinic.owner.Pe...,77,20,16
3,org.springframework.samples.petclinic.owner.Pet,71,22,19
4,org.springframework.samples.petclinic.owner.Vi...,49,31,12


# Load data into Neo4J

Esablish connection to Neo4j graph database

In [10]:
from py2neo import Graph
graph = Graph("http://localhost:7474", password="neo4j")
graph

Graph('http://localhost:7474')

Clean data from previous run

In [11]:
query="""
   MATCH (n:JType)
   MATCH (m:JMeasure)
   DETACH DELETE n, m
"""
graph.run(query)

(No data)

## jdeps

In [12]:
petclinic_deps.head()

,raw,from,to,type,name
867,org.springframework.samples.petclinic.PetCl...,org.springframework.samples.petclinic.PetClini...,java.lang.Class,,PetClinicApplication
868,org.springframework.samples.petclinic.PetCl...,org.springframework.samples.petclinic.PetClini...,java.lang.Object,,PetClinicApplication
869,org.springframework.samples.petclinic.PetCl...,org.springframework.samples.petclinic.PetClini...,java.lang.String,,PetClinicApplication
870,org.springframework.samples.petclinic.PetCl...,org.springframework.samples.petclinic.PetClini...,org.springframework.boot.SpringApplication,not found,PetClinicApplication
871,org.springframework.samples.petclinic.PetCl...,org.springframework.samples.petclinic.PetClini...,org.springframework.boot.autoconfigure.SpringB...,not found,PetClinicApplication


In [13]:
query="""
    UNWIND {deps_data} as dep
    CREATE
        (t:JType)
    SET
        t.fqn = dep.from,
        t.name = dep.name
    RETURN t.fqn, t.name
"""

result = graph.run(query, deps_data=petclinic_deps[['from', 'name']].drop_duplicates().to_dict(orient='records')).data()
pd.DataFrame(result).head()

,t.fqn,t.name
0,org.springframework.samples.petclinic.PetClini...,PetClinicApplication
1,org.springframework.samples.petclinic.model.Ba...,BaseEntity
2,org.springframework.samples.petclinic.model.Na...,NamedEntity
3,org.springframework.samples.petclinic.model.Pe...,Person
4,org.springframework.samples.petclinic.owner.Owner,Owner


Create index for `fqn" for faster queries

In [14]:
query="""
  CREATE INDEX ON :J(fqn)
"""
graph.run(query)

(No data)

In [15]:
query="""
    UNWIND {deps_data} as dep
    MATCH (from:JType {fqn : dep.from})
    MATCH (to:JType {fqn: dep.to})
    MERGE (from)-[:DEPENDS_ON]->(to)
    RETURN from.fqn, to.fqn
"""

result = graph.run(query, deps_data=petclinic_deps.to_dict(orient='records')).data()
pd.DataFrame(result).head()

,from.fqn,to.fqn
0,org.springframework.samples.petclinic.model.Na...,org.springframework.samples.petclinic.model.Ba...
1,org.springframework.samples.petclinic.model.Pe...,org.springframework.samples.petclinic.model.Ba...
2,org.springframework.samples.petclinic.owner.Owner,org.springframework.samples.petclinic.model.Pe...
3,org.springframework.samples.petclinic.owner.Owner,org.springframework.samples.petclinic.owner.Pet
4,org.springframework.samples.petclinic.owner.Ow...,org.springframework.samples.petclinic.owner.Owner


## JaCoCo coverage data

In [16]:
coverage.head()

,PACKAGE,CLASS,LINE_MISSED,LINE_COVERED,lines,ratio,fqn
0,org.springframework.samples.petclinic,PetclinicInitializer,0,24,24,1.0,org.springframework.samples.petclinic.Petclini...
1,org.springframework.samples.petclinic.model,NamedEntity,1,4,5,0.8,org.springframework.samples.petclinic.model.Na...
2,org.springframework.samples.petclinic.model,Specialty,0,1,1,1.0,org.springframework.samples.petclinic.model.Sp...
3,org.springframework.samples.petclinic.model,PetType,0,1,1,1.0,org.springframework.samples.petclinic.model.Pe...
4,org.springframework.samples.petclinic.model,Vets,4,0,4,0.0,org.springframework.samples.petclinic.model.Vets


In [17]:
query="""
    UNWIND {coverage_data} as coverage
    MATCH (t:JType {fqn : coverage.fqn})
    MERGE (t)-[:HAS_JMEASURE]->(m)
    SET 
        m:JMeasure:JCoverage,
        m.ratio = coverage.ratio
    RETURN t.fqn as fqn, m.ratio as ratio
"""

result = graph.run(query, coverage_data=coverage.to_dict(orient='records')).data()
pd.DataFrame(result).head()

,fqn,ratio
0,org.springframework.samples.petclinic.model.Na...,0.8
1,org.springframework.samples.petclinic.model.Ba...,1.0
2,org.springframework.samples.petclinic.model.Pe...,1.0


## cloc data

In [18]:
loc.head()

,fqn,code,comment,blank
0,org.springframework.samples.petclinic.owner.Ow...,96,31,18
1,org.springframework.samples.petclinic.owner.Owner,94,33,23
2,org.springframework.samples.petclinic.owner.Pe...,77,20,16
3,org.springframework.samples.petclinic.owner.Pet,71,22,19
4,org.springframework.samples.petclinic.owner.Vi...,49,31,12


In [19]:
query="""
    UNWIND {cloc_data} as loc
    MATCH (t:JType {fqn : loc.fqn})
    SET
        t.lines = loc.code,
        t.comments = loc.comment,
        t.blanks = loc.blank
    RETURN t.fqn, t.name, t.lines, t.comments, t.blanks
"""

result = graph.run(query, cloc_data=loc.to_dict(orient='records')).data()
pd.DataFrame(result).head()

,t.fqn,t.name,t.lines,t.comments,t.blanks
0,org.springframework.samples.petclinic.owner.Ow...,OwnerController,96,31,18
1,org.springframework.samples.petclinic.owner.Owner,Owner,94,33,23
2,org.springframework.samples.petclinic.owner.Pe...,PetController,77,20,16
3,org.springframework.samples.petclinic.owner.Pet,Pet,71,22,19
4,org.springframework.samples.petclinic.owner.Vi...,VisitController,49,31,12


# Check data

## Validate Nodes

In [20]:
query="""
   MATCH (n:JType)
   RETURN n.fqn
"""

result = graph.run(query).data()
pd.DataFrame(result)

,n.fqn
0,org.springframework.samples.petclinic.owner.Ow...
1,org.springframework.samples.petclinic.owner.Pet
2,org.springframework.samples.petclinic.owner.Pe...
3,org.springframework.samples.petclinic.owner.Pe...
4,org.springframework.samples.petclinic.owner.Pe...
5,org.springframework.samples.petclinic.owner.Pe...
6,org.springframework.samples.petclinic.owner.Pe...
7,org.springframework.samples.petclinic.owner.Vi...
8,org.springframework.samples.petclinic.system.C...
9,org.springframework.samples.petclinic.system.C...


In [21]:
query="""
   MATCH (n:JType)-[:HAS_JMEASURE]->(m:JMeasure)
   RETURN n.fqn, n.lines, m.ratio
"""

result = graph.run(query).data()
pd.DataFrame(result)

,n.fqn,n.lines,m.ratio
0,org.springframework.samples.petclinic.model.Na...,18,0.8
1,org.springframework.samples.petclinic.model.Ba...,21,1.0
2,org.springframework.samples.petclinic.model.Pe...,25,1.0
